In [1]:
from langchain_core.messages import AIMessage, HumanMessage, SystemMessage
from langchain_google_genai import ChatGoogleGenerativeAI

In [2]:
class Agent:
    def __init__(self, system=""):
        self.system = system
        self.messages = []
        if self.system:
            self.messages.append(SystemMessage(content=system))
        self.model = ChatGoogleGenerativeAI(model="gemini-1.5-flash", temperature=0)

    def __call__(self, message):
        self.messages.append(HumanMessage(content=message))
        result = self.execute()
        self.messages.append(AIMessage(content=result))
        return result

    def execute(self):
        response = self.model.invoke(self.messages)
        return response.content

In [3]:
prompt = """
    You run in a loop of Thought, Action, PAUSE, Observation.
    At the end of the loop you output an Answer.
    
    Use Thought to describe your thoughts about the question you have been asked.
    Use Action to run one of the actions available to you - then return PAUSE.
    Observation will be the result of running those actions.
    
    Your available actions are:
    get_apple_cost: <city>
        Returns the cost of apples per kg in the given city.
    
    calculate: <expression>
        Runs a Python calculation and returns the result. Use floating point syntax if necessary.
    
    Example session:
    Question: What is the average cost of apples in Berlin, Paris, and Rome?
    Thought: I need to get the cost of apples in each city first.
    Action: get_apple_cost: Berlin
    PAUSE
    
    You will be called again with:
    Observation: Apples cost 3.2 €/kg in Berlin
    Thought: Now I will get the price in Paris
    Action: get_apple_cost: Paris
    PAUSE
    
    Observation: Apples cost 2.8 €/kg in Paris
    Thought: Now I will get the price in Rome
    Action: get_apple_cost: Rome
    PAUSE
    
    Observation: Apples cost 3.0 €/kg in Rome
    Thought: I will now calculate the average of the three prices: (3.2 + 2.8 + 3.0) / 3
    Action: calculate: (3.2 + 2.8 + 3.0) / 3
    PAUSE
    
    Observation: 3.0
    Answer: The average cost of apples in Berlin, Paris, and Rome is 3.0 €/kg
""".strip()

In [4]:
def get_apple_cost(city):
    prices = {"Berlin": 3.2, "Paris": 2.8, "Rome": 3.0, "Madrid": 2.5, "London": 3.5}
    if city in prices:
        return f"Apples cost {prices[city]} €/kg in {city}"
    else:
        return "City not found"


def calculate(x):
    return eval(x)


known_actions = {
    "get_apple_cost": get_apple_cost,
    "calculate": calculate,
}

In [5]:
agent = Agent(prompt)

In [6]:
answer = agent("How much do apples cost in Berlin?")
print(answer)

Thought: I need to get the cost of apples in Berlin.
Action: get_apple_cost: Berlin
PAUSE


In [7]:
result = get_apple_cost("Berlin")
result

'Apples cost 3.2 €/kg in Berlin'

In [8]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: Apples cost 3.2 €/kg in Berlin'

In [9]:
agent(next_prompt)

'Answer: Apples cost 3.2 €/kg in Berlin.'

In [10]:
agent.messages

[SystemMessage(content='You run in a loop of Thought, Action, PAUSE, Observation.\n    At the end of the loop you output an Answer.\n\n    Use Thought to describe your thoughts about the question you have been asked.\n    Use Action to run one of the actions available to you - then return PAUSE.\n    Observation will be the result of running those actions.\n\n    Your available actions are:\n    get_apple_cost: <city>\n        Returns the cost of apples per kg in the given city.\n\n    calculate: <expression>\n        Runs a Python calculation and returns the result. Use floating point syntax if necessary.\n\n    Example session:\n    Question: What is the average cost of apples in Berlin, Paris, and Rome?\n    Thought: I need to get the cost of apples in each city first.\n    Action: get_apple_cost: Berlin\n    PAUSE\n\n    You will be called again with:\n    Observation: Apples cost 3.2 €/kg in Berlin\n    Thought: Now I will get the price in Paris\n    Action: get_apple_cost: Paris\

In [11]:
agent = Agent(prompt)

In [12]:
question = "What’s the average price of apples in Berlin, Paris, and Rome?"
print(agent(question))

Thought: I need to get the price of apples in each city.
Action: get_apple_cost: Berlin
PAUSE


In [13]:
result = get_apple_cost("Berlin")
result

'Apples cost 3.2 €/kg in Berlin'

In [14]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: Apples cost 3.2 €/kg in Berlin'

In [15]:
print(agent(next_prompt))

Thought: Now I need the price of apples in Paris.
Action: get_apple_cost: Paris
PAUSE


In [16]:
result = get_apple_cost("Paris")
result

'Apples cost 2.8 €/kg in Paris'

In [17]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: Apples cost 2.8 €/kg in Paris'

In [18]:
print(agent(next_prompt))

Thought: Now I need the price of apples in Rome.
Action: get_apple_cost: Rome
PAUSE


In [19]:
result = get_apple_cost("Rome")
result

'Apples cost 3.0 €/kg in Rome'

In [20]:
next_prompt = "Observation: {}".format(result)
next_prompt

'Observation: Apples cost 3.0 €/kg in Rome'

In [21]:
print(agent(next_prompt))

Thought: I will now calculate the average price.
Action: calculate: (3.2 + 2.8 + 3.0) / 3
PAUSE


In [22]:
next_prompt = "Observation: {}".format(eval("(3.2 + 2.8 + 3.0) / 3"))
print(next_prompt)

Observation: 3.0


In [23]:
print(agent(next_prompt))

Answer: The average price of apples in Berlin, Paris, and Rome is 3.0 €/kg.


In [24]:
agent.messages

[SystemMessage(content='You run in a loop of Thought, Action, PAUSE, Observation.\n    At the end of the loop you output an Answer.\n\n    Use Thought to describe your thoughts about the question you have been asked.\n    Use Action to run one of the actions available to you - then return PAUSE.\n    Observation will be the result of running those actions.\n\n    Your available actions are:\n    get_apple_cost: <city>\n        Returns the cost of apples per kg in the given city.\n\n    calculate: <expression>\n        Runs a Python calculation and returns the result. Use floating point syntax if necessary.\n\n    Example session:\n    Question: What is the average cost of apples in Berlin, Paris, and Rome?\n    Thought: I need to get the cost of apples in each city first.\n    Action: get_apple_cost: Berlin\n    PAUSE\n\n    You will be called again with:\n    Observation: Apples cost 3.2 €/kg in Berlin\n    Thought: Now I will get the price in Paris\n    Action: get_apple_cost: Paris\